In [ ]:
import tensorflow as tf
import keras
%load_ext autoreload
%autoreload 2
from grad import *
from model import *
import numpy as np


# I check autograd libraries

#Holy debugging time
def np_number(k):
    return np.array([Number(i) for i in range(k)])
m = Number(2)
a = (Number(3) + m) * 2.1 - (Number(9) - m) / (Number(3) - 8 +4)
print(topo_sort(a))
print(old_sort(a))
print(gpt_sort(a))

In [ ]:
x1 = Number(1)
x2 = Number(2)
x3 = Number(3)
x4 = Number(4)
x5 = Number(5)

y = (x1+x2)*x5 - x3
top_sorted = topo_sort(y)
print(top_sorted)

y.null_gradients()

for num in top_sorted:
    num.backprop_single()
    
print([(i.grad, i) for i in top_sorted])

y.null_gradients()
y.backprop()

print( [(i.grad, i) for i in top_sorted], "key")

# data getting

In [ ]:

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data(path="mnist.npz", )
indices = np.arange(len(x_train))
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

# Keras Model

In [ ]:
def get_flat_dataset(raw_x, raw_y):
  raw_x = raw_x / 255
  dataset = tf.data.Dataset.from_tensor_slices((raw_x.flatten().reshape(-1, 28*28), raw_y))
  return dataset

In [ ]:
flat_train = get_flat_dataset(x_train[:500], y_train[:500])
flat_test = get_flat_dataset(x_test[:500], y_test[:500])

In [ ]:
inputs = keras.Input(shape=((28* 28,)))
x = keras.layers.Dense(4)(inputs)
x = keras.layers.Dense(8)(x)

outputs = keras.layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", #labels appear to be from one to ten so i think this the right loss
              optimizer="rmsprop",
              metrics=['accuracy'])


history = model.fit(flat_train.take(1000).batch(4, drop_remainder=True).prefetch(tf.data.AUTOTUNE), epochs=20,
                      validation_data=(flat_test.batch(4, drop_remainder = True).prefetch(tf.data.AUTOTUNE)),
                      )


# My model


In [ ]:
def batch(x, y, batch_size=32):
    if len(x) % batch_size != 0:
        x = x[:batch_size * (len(x)//batch_size)]
        y=y[:batch_size*(len(x)//batch_size)]
    return np.array_split(x, len(x) / batch_size, axis=0), np.array_split(y, len(y)/batch_size, axis=0)

In [ ]:
def fix_data(x, y):
    x = x.reshape(x.shape[0], 28*28)/255
    test = np.zeros((x.shape[0], 10))
    test[np.arange(x.shape[0]),y] = 1
    return (x, test)

In [ ]:
fixed_x, fixed_y = fix_data(x_train[:500], y_train[:500])
b_x , b_y = batch(fixed_x, fixed_y, 4)


In [ ]:
my_model = Model(28*28, 10, [4, 8])
datas = []
for _epoch in range(10):
    datas.append(my_model.train_epoch(b_x, b_y, lr=10e-1))

In [ ]:
print(datas)

In [ ]:
a = Number(2)
print(isinstance(a,  (Number, int, float, np.generic)))
Number(1) * Number(2)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

fig = plt.figure(figsize=(10, 7))
pic = 1
for i, img in enumerate(x_test[:10]):
  plt.subplot(2, 5, pic)
  plt.axis('off')
  predicted = my_model.fd(img.flat)
  keras_pred = model.predict(img.reshape(1, 28*28))
  plt.title(f"T {y_test[i]} mine {np.argmax(predicted)} keras {np.argmax(keras_pred)} ")
  plt.imshow(img)
  pic+= 1
plt.show()
#60% acc. Considering this is from nearly scratch not terrible 

# Comparison!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

fig = plt.figure(figsize=(10, 7))
pic = 1
for i, img in enumerate(x_test[:10]):
  plt.subplot(2, 5, pic)
  plt.axis('off')
  predicted = my_model.fd(img.flat)
  keras_pred = model.predict(img.reshape(1, 28*28))
  plt.title(f"T {y_test[i]} mine {np.argmax(predicted)} keras {np.argmax(keras_pred)} ")
  plt.imshow(img)
  pic+= 1
plt.show()
#60% acc. Considering this is from nearly scratch not terrible 

# Reasons my model is kind of terrible and how to improve.
1. Slow. Still not 100% sure where the computation comes from. (but ima guess backprop) To do: Go figure that out.
2. Too little training data (This will be fixed with slow otherwise it would take days to train)
3. No batching. Need to figure that out too. (and how backprop works with batching)
4. No rmsprop. I don't even know where to start on that one.
5. No validation data. Yet.


More notes:
Log things you've changed. esp when theres a gazillion interlocking things, FIX ONE AT A TIME
Have you tried 